In [0]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tensorflow as tf
import tflearn
import random

In [0]:
!pip install tflearn

    100% |████████████████████████████████| 102kB 2.3MB/s 
  Running setup.py bdist_wheel for tflearn ... - \ done
  Stored in directory: /content/.cache/pip/wheels/fb/06/72/0478c938ca315c6fddcce8233b80ec91a115ce4496a27e8c90
Successfully built tflearn


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import json
with open('drive/Colab Notebooks/chatbot/intents.json') as json_data:
    intents = json.load(json_data)

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
words=[]
documents=[]
classes=[]
ignore_words=['?']

stemmer = LancasterStemmer()

# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
     # tokenize each word in the sentence
         w=nltk.word_tokenize(pattern)
     # add to our words list
         words.extend(w)
    # add to documents in our corpus
         documents.append((w, intent['tag']))
     # add to our classes list
         if intent['tag'] not in classes:
             classes.append(intent['tag'])
        

# stem and lower each word and remove duplicates
words=[stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))

# remove duplicates
classes=sorted(list(set(classes)))

In [0]:
print (len(documents), "documents",documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)


25 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Namaste'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['TELL', 'ME', 'ABOUT', 'TAKING', 'SOIL', 'SAMPLE'], 'Soil Testing'), (['How', 'to', 'take', 'soil', 'sample', '?'], 'Soil Testing'), (['What', 'is', 'the', 'technique', 'used', 'in', 'taking', 'soil', 'sample'], 'Soil Testing'), (['TELL', 'ME', 'SOMETHING', 'ABOUT', 'NUTRIENT', 'MANAGEMENT', '?'], 'Nutrient Management'), (['What', 'are', 'some', 'ways', 'to', 'improve', 'nutrients', 'in', 'soil'], 'Nutrient Management'), (['How', 'to', 'manage', 'nutrients'], 'Nutrient Management'), (['TELL', 'ME', 'HOW', 'TO', 'CONTROL', 'INSECT', 'IN', 'WHEAT'], 'Plant Protection'), (['What', 'are', 'some', 'ways', 'to', 'co

In [0]:
# create our training data
training=[]
output=[]

# create an empty array for our output
output_empty=[0]*len(classes)

# training set, bag of words for each sentence
for doc in documents:
    bag=[]
     # list of tokenized words for the pattern
    pattern_words=doc[0]
    pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
     # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
     # output is a '0' for each tag and '1' for current tag
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])
    
# shuffle our features and turn into np.array.
random.shuffle(training)
training=np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


In [0]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('drive/Colab Notebooks/chatbot/model.tflearn')

Training Step: 3999  | total loss: 0.78752 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.78752 - acc: 0.8998 -- iter: 24/25
Training Step: 4000  | total loss: 0.71449 | time: 0.019s
| Adam | epoch: 1000 | loss: 0.71449 - acc: 0.9098 -- iter: 25/25
--
INFO:tensorflow:/content/drive/Colab Notebooks/chatbot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
# save all of our data structures
import pickle
pickle.dump({'words':words,'classes':classes,'train_x':train_x,'train_y':train_y},open('drive/Colab Notebooks/chatbot/training_data','wb'))